In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import pandas as pd
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from scipy import stats
from libpysal.weights import lat2W
from scipy.spatial.distance import jensenshannon
from scipy import stats
from esda.moran import Moran

In [ ]:
def open_data():
    #Open the datasets
    factual = xr.open_mfdataset("factual/*.nc")
    cfl = xr.open_mfdataset("eth_cfl/*.nc", join='inner', compat='override')
    
    factual = factual.reduce(np.nansum, dim='expver',keep_attrs=True)
    cfl["lon"] = np.arange(-180,180,2.5)
    factual = factual.rename({"latitude":"lat","longitude":"lon"})
    cfl = cfl.sel(lat=slice(0,60),lon=slice(-80,20))
    
    #Regrid the factual dataset to be the counterfactual's granularity
    ds_out = xr.Dataset(
        {
            "lat": np.array(cfl["lat"]),
            "lon": np.array(cfl["lon"]),
        }
    )
    regridder = xe.Regridder(factual, ds_out, "bilinear")
    factual = regridder(factual)
    #factual = factual.isel(time=slice(0,864)) 
    factual = factual.isel(time=slice(0,732)) # if monthly data
    
    pred_df = pd.read_csv("yearly_activity.csv")
    pred_df = pred_df.loc[pred_df['Year'] >= 1950]
    ace_raw = pred_df['Accumulated Cyclone Energy']
    ace = np.array(ace_raw)
    
    return factual,cfl,ace

In [ ]:
factual, cfl,ace = open_data()
cfl = xr.open_mfdataset("eth_cfl/*.nc", join='inner', compat='override')
cfl["lon"] = np.arange(-180,180,2.5)
cfl = cfl.sel(lat=slice(0,60),lon=slice(-80,20))
cfl = cfl.drop("time_bnds")
cfl = cfl.assign(ua_50000=cfl["ua"].sel(plev=50000))
cfl = cfl.assign(va_50000=cfl["va"].sel(plev=50000))
cfl = cfl.drop("ua")
cfl = cfl.drop("va")
cfl = cfl.drop("plev")

In [ ]:
sst_cpl = cfl['SST_cpl'].to_numpy().reshape(120,2624)
ice_cov = cfl['ice_cov'].to_numpy().reshape(120,2624)
psl = cfl['psl'].to_numpy().reshape(120,2624)
ua_50k = cfl['ua_50000'].to_numpy().reshape(120,2624)
va_50k = cfl['va_50000'].to_numpy().reshape(120,2624)

sst = factual['sst'].to_numpy().reshape(732,2624)
ice = factual['siconc'].to_numpy().reshape(732,2624)
p = factual['msl'].to_numpy().reshape(732,2624)
ua = factual['u10'].to_numpy().reshape(732,2624)
va = factual['v10'].to_numpy().reshape(732,2624)

In [ ]:
plt.hist(sst_cpl)
plt.show()
plt.hist(sst)
plt.show()

In [ ]:
plt.hist(ice_cov)
plt.show()
plt.hist(ice)
plt.show()

In [ ]:
plt.hist(psl)
plt.show()
plt.hist(p)
plt.show()

In [ ]:
plt.hist(ua_50k)
plt.show()
plt.hist(ua)
plt.show()

In [ ]:
plt.hist(va_50k)
plt.show()
plt.hist(va)
plt.show()

In [ ]:
sst = sst.reshape((732,64,41))
ice = ice.reshape((732,64,41))
p = p.reshape((732,64,41))
ua = ua.reshape((732,64,41))
va = va.reshape((732,64,41))

In [ ]:
sst_cpl = sst_cpl.reshape((120,64,41))
ice_cov = ice_cov.reshape((120,64,41))
psl = psl.reshape((120,64,41))
ua_50k = ua_50k.reshape((120,64,41))
va_50k = va_50k.reshape((120,64,41))

In [ ]:
w = lat2W(sst[0].shape[0], sst[0].shape[1])

In [ ]:
mi = Moran(sst[0], w)

In [ ]:
print(mi.I) 
print(mi.p_norm)

In [ ]:
m_l = []
for i in range(732):
    print(i)
    w = lat2W(sst[i].shape[0], sst[i].shape[1])
    mi = Moran(sst[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for SST: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(732):
    print(i)
    w = lat2W(ice[i].shape[0], ice[i].shape[1])
    mi = Moran(ice[i],w)
    m_l.append(mi.I)
print('Average SST Moran\'s I measure for sea ice: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(732):
    print(i)
    w = lat2W(p[i].shape[0], p[i].shape[1])
    mi = Moran(p[i],w)
    m_l.append(mi.I)
print('Average SST Moran\'s I measure for pressure: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(732):
    print(i)
    w = lat2W(ua[i].shape[0], ua[i].shape[1])
    mi = Moran(ua[i],w)
    m_l.append(mi.I)
print('Average SST Moran\'s I measure for u-wind: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(732):
    print(i)
    w = lat2W(va[i].shape[0], va[i].shape[1])
    mi = Moran(va[i],w)
    m_l.append(mi.I)
print('Average SST Moran\'s I measure for v-wind: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(120):
    print(i)
    w = lat2W(sst_cpl[i].shape[0], sst_cpl[i].shape[1])
    mi = Moran(sst_cpl[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for CTF-SST: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(120):
    print(i)
    w = lat2W(ice_cov[i].shape[0], ice_cov[i].shape[1])
    mi = Moran(ice_cov[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for CTF-sea ice: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(120):
    print(i)
    w = lat2W(psl[i].shape[0], psl[i].shape[1])
    mi = Moran(psl[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for CTF-pressure: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(120):
    print(i)
    w = lat2W(ua_50k[i].shape[0], ua_50k[i].shape[1])
    mi = Moran(ua_50k[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for CTF-U_wind: ' + str(np.average(m_l)))

In [ ]:
m_l = []
for i in range(120):
    print(i)
    w = lat2W(va_50k[i].shape[0], va_50k[i].shape[1])
    mi = Moran(va_50k[i],w)
    m_l.append(float(mi.I))
print('Average SST Moran\'s I measure for CTF-V_wind: ' + str(np.average(m_l)))

In [ ]:
sst_pdf = stats.gaussian_kde(sst.flatten())
ctf_sst_pdf = stats.gaussian_kde(sst_cpl.flatten())
ice_pdf = stats.gaussian_kde(ice.flatten())
ctf_ice_pdf = stats.gaussian_kde(ice_cov.flatten())
p_pdf = stats.gaussian_kde(p.flatten())
ctf_p_pdf = stats.gaussian_kde(psl.flatten())
ua_pdf = stats.gaussian_kde(ua.flatten())
ctf_ua_pdf = stats.gaussian_kde(ua_50k.flatten())
va_pdf = stats.gaussian_kde(va.flatten())
ctf_va_pdf = stats.gaussian_kde(va_50k.flatten())

In [ ]:
print(np.min(sst))
print(np.max(sst))
print(np.min(sst_cpl))
print(np.max(sst_cpl))
print(np.min(ice))
print(np.max(ice))
print(np.min(ice_cov))
print(np.max(ice_cov))
print(np.min(p))
print(np.max(p))
print(np.min(psl))
print(np.max(psl))
print(np.min(ua))
print(np.max(ua))
print(np.min(ua_50k))
print(np.max(ua_50k))
print(np.min(va))
print(np.max(va))
print(np.min(va_50k))
print(np.max(va_50k))
sst_min = min(np.min(sst),np.min(sst_cpl))
sst_max = min(np.max(sst),np.max(sst_cpl))
ice_min = min(np.min(ice),np.min(ice_cov))
ice_max = min(np.max(ice),np.max(ice_cov))
p_min = min(np.min(p),np.min(psl))
p_max = min(np.max(p),np.max(psl))
ua_min = min(np.min(ua),np.min(ua_50k))
ua_max = min(np.max(ua),np.max(ua_50k))
va_min = min(np.min(va),np.min(va_50k))
va_max = min(np.max(va),np.max(va_50k))

In [ ]:
X_sst = np.linspace(sst_min,sst_max)
X_ice = np.linspace(ice_min,ice_max)
X_p = np.linspace(p_min,p_max)
X_ua = np.linspace(ua_min,ua_max)
X_va = np.linspace(va_min,va_max)

In [ ]:
y_sst_f = sst_pdf(X_sst)
y_sst_ctf = ctf_sst_pdf(X_sst)
y_ice_f = ice_pdf(X_ice)
y_ice_ctf = ctf_ice_pdf(X_ice)
y_p_f = p_pdf(X_p)
y_p_ctf = ctf_p_pdf(X_p)
y_ua_f = ua_pdf(X_ua)
y_ua_ctf = ctf_ua_pdf(X_ua)
y_va_f = va_pdf(X_va)
y_va_ctf = ctf_va_pdf(X_va)

In [ ]:
print('JS SST: ' + str(jensenshannon(y_sst_f,y_sst_ctf)))
print('JS ice: ' + str(jensenshannon(y_ice_f,y_ice_ctf)))
print('JS pressure: ' + str(jensenshannon(y_p_f,y_p_ctf)))
print('JS ua: ' + str(jensenshannon(y_ua_f,y_ua_ctf)))
print('JS va: ' + str(jensenshannon(y_va_f,y_va_ctf)))